# TipTrack CNN Training

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.metrics import categorical_crossentropy, binary_crossentropy
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, RandomFlip, RandomRotation
from keras.utils.vis_utils import plot_model
from keras.callbacks import Callback
import visualkeras
# from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomBrightness

# Custom python class for preparing the dataset. This will prevent the notebook from being too messy
from prepare_dataset import PrepareDataset

# Ignore CPU warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
IMG_SIZE = 48  # The expected size of the input images
TARGET_FOLDER = 'training_images/2023-02-24'  # The folder containing the new images

## Prepare the training and test dataset

In [ ]:
prepare_dataset = PrepareDataset()

train_X, train_label, test_X, test_label = prepare_dataset.get_dataset(TARGET_FOLDER, IMG_SIZE)

## Init Keras Model

**Breakdown of the current architecture:**

The model is defined as a Sequential object, meaning layers will be added in a sequential manner.

The first layer added is a Conv2D layer with 64 filters, each with a kernel size of 3x1, and a linear activation function. The input shape is (48, 48, 1), which means the input is a grayscale image with dimensions 48x48. The padding is set to 'same', which means the output will have the same spatial dimensions as the input.

The next layer is another Conv2D layer with 64 filters, this time with a kernel size of 1x3 and also with a linear activation function. The padding is set to 'same'.

A MaxPooling2D layer is added with a pool size of 2x2 and padding set to 'same'. This layer reduces the spatial dimensions of the output by a factor of 2.

Two more pairs of Conv2D and MaxPooling2D layers are added, each with a lower number of filters (32) but larger kernel sizes (3x1 and 1x3). The padding is set to 'same' for both convolutional layers, and the pool size is again 2x2 with padding set to 'same' for both pooling layers. A Dropout layer with rate 0.2 is added after the second pair of convolutional and pooling layers.

A Flatten layer is added to flatten the output of the previous layer into a 1D array.

Four Dense layers with 128 units each and a ReLU activation function are added in sequence.

Another Dropout layer with rate 0.2 is added after the fifth Dense layer.

A final Dense layer with 2 units (one for each possible class) and a softmax activation function is added.

The model is compiled using categorical_crossentropy as the loss function, the Adam optimizer, and accuracy as the metric to monitor during training.

In [ ]:
def init_keras_model():
    # conv linear
    activation = 'relu'
    activation_conv = 'LeakyReLU'  # LeakyReLU
    num_neurons = 64 # 128

    # Define model structure
    model = Sequential()

    # model.add(data_augmentation_new)
    model.add(Conv2D(64, kernel_size=(3, 1), activation=activation_conv, input_shape=(IMG_SIZE, IMG_SIZE, 1), padding='same'))
    model.add(Conv2D(64, (1, 3), activation=activation_conv, padding='same'))

    # Reduce the spatial dimensions of the output by a factor of 2
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    model.add(Conv2D(32, (3, 1), activation=activation_conv, padding='same'))
    model.add(Conv2D(32, (1, 3), activation=activation_conv, padding='same'))

    # Reduce the spatial dimensions of the output by a factor of 2
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    #model.add(Dropout(0.2))

    model.add(Conv2D(32, (3, 3), activation=activation_conv, padding='same'))
    # Reduce the spatial dimensions of the output by a factor of 2
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    model.add(Dropout(0.5))

    # model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
    # model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    model.add(Flatten())

    # trial and error: linear performs much better than ReLU and sigmoid
    model.add(Dense(num_neurons, activation=activation))
    # model.add(Dense(num_neurons, activation=activation))
    # model.add(Dense(num_neurons, activation=activation))
    # model.add(Dense(num_neurons, activation=activation))

    model.add(Dropout(0.5))

    model.add(Dense(num_neurons, activation=activation))
    # model.add(Dense(64, activation=activation))
    # model.add(Dense(64, activation=activation))
    # model.add(Dense(128, activation='linear'))
    # model.add(Dense(128, activation='linear'))

    # Classifier
    model.add(Dense(2, activation='softmax'))

    model.compile(loss=categorical_crossentropy, optimizer="adam", metrics=['accuracy'])

    # reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)

    model.summary()
    
    # plot_model(model)
    visualkeras.layered_view(model, legend=True)
    
    return model

In [ ]:
# https://medium.com/@mayankverma05032001/binary-classification-using-convolution-neural-network-cnn-model-6e35cdf5bdbb

def init_keras_model_2():
    model = Sequential()
    model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
    
    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    
    # TODO: Why here 2 and not 1 for binary classification?
    model.add(Dense(2, activation='sigmoid'))
    
    model.compile(loss=binary_crossentropy, optimizer="adam", metrics=['accuracy'])

    # reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)

    model.summary()
    
    # plot_model(model)
    visualkeras.layered_view(model, legend=True)
    
    return model
    
    

In [ ]:
# def init_keras_model_2():
#     model = Sequential()
#         # Note the input shape is the desired size of the image 200x200 with 3 bytes color
#         # This is the first convolution
#     model.add(Conv2D(16, (3,3), activation='relu', input_shape=(48, 48, 1)))
#     model.add(MaxPooling2D(2, 2))
#         # The second convolution
#     model.add(Conv2D(32, (3,3), activation='relu'))
#     model.add(MaxPooling2D(2,2))
#         # The third convolution
#     model.add(Conv2D(64, (3,3), activation='relu'))
#     model.add(MaxPooling2D(2,2))
#         # The fourth convolution
#     model.add(Conv2D(64, (3,3), activation='relu'))
#     model.add(MaxPooling2D(2,2))
#         # # The fifth convolution
#     model.add(Conv2D(64, (3,3), activation='relu'))
#     model.add(MaxPooling2D(2,2))
#         # Flatten the results to feed into a DNN
#     model.add(Flatten())
#         # 512 neuron hidden layer
#     model.add(Dense(128, activation='relu'))
#         # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('dandelions') and 1 for the other ('grass')
#     model.add(Dense(1, activation='sigmoid'))
        
#     model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics='accuracy')
    
#     return model

## Train Model

In [ ]:
# Custom Keras callback class to plot the data after each epoch
class plot_callback(Callback):
    
    def on_epoch_end(self, epoch, logs={}):
        global history
        global total_epochs
        global loss
        global val_loss
        global accuracy
        global val_accuracy
        
        total_epochs += 1
        
        loss.append(logs.get('loss'))
        val_loss.append(logs.get('val_loss'))
        accuracy.append(logs.get('accuracy'))
        val_accuracy.append(logs.get('val_accuracy'))
        
        # if total_epochs > 1:
            
        plt.plot(accuracy, 'blue', label='Training accuracy', marker="o", markersize=5)
        plt.plot(val_accuracy, 'green', label='Validation accuracy', marker="o", markersize=5)
        plt.plot(loss, 'red', label='Loss', marker="o", markersize=5)
        plt.plot(val_loss, 'orange', label='Validation loss', marker="o", markersize=5)

        plt.title('Training and Validation accuracy/loss')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy/Loss')
        plt.xticks(np.arange(0, total_epochs + 1, 1))
        plt.ylim(0, 1)
        plt.legend()
        plt.show()

        

        #plt.title('Training and Validation loss')
        #plt.xlabel('Epochs')
        #plt.ylabel('Loss')
        #plt.xticks(np.arange(0, total_epochs + 1, 1))
        #plt.ylim(0, 1)
        #plt.legend()
        #plt.show()


In [ ]:
total_epochs= 0
loss = []
val_loss = []
accuracy = []
val_accuracy = []

model = init_keras_model_2()

history = model.fit(
    x=train_X,
    y=train_label,
    batch_size=128,
    epochs=10,
    verbose=1,
    shuffle=True,  # Try shuffling data here after each epoch
    validation_data=(test_X, test_label),
    callbacks=[plot_callback()]
)

## Save Model

In [ ]:
MODEL_NAME = 'model_2023_019'  # How the output model should be named at the end

if MODEL_NAME in next(os.walk('.'))[1]:
    print('Warning: There already exists a model called "{}"'.format(MODEL_NAME))
    print('Make sure you want to overwrite it before it gets saved!')
else:
    model.save(MODEL_NAME)

### Possible problems (ChatGPT)

Without knowing the specifics of the dataset and the task at hand, it's difficult to say whether this model is appropriate or not. However, there are some general points to consider:

1.  The model has a relatively large number of parameters, which can increase the risk of overfitting on small datasets. In particular, having five consecutive Dense layers with 128 units each may be unnecessary for some tasks and can lead to overfitting if the dataset is not large enough.
2.  The use of linear activation functions in the convolutional layers is not common in modern architectures. In most cases, rectified linear unit (ReLU) or its variants are used as activation functions in convolutional layers. However, it's possible that linear activation functions are appropriate for the specific task at hand.
3.  The Dropout rate of 0.2 used in this model may be too low for some datasets, especially if the model is prone to overfitting. In general, it's recommended to start with a higher rate of Dropout (e.g. 0.5) and then tune it based on the performance of the model on the validation set.
4.  There is no use of Batch Normalization, which is a technique used to improve the training of deep neural networks. Batch Normalization can help improve the stability of the network during training and improve the overall accuracy of the model. However, it's possible that the specific task at hand does not require Batch Normalization.
5.  Lastly, it's important to ensure that the dataset is properly preprocessed before training the model. This includes tasks such as scaling the pixel values to a common range, converting images to grayscale or RGB, and splitting the dataset into training, validation, and test sets.

To improve the prediction of a neural network model, here are some suggestions that you can try:

1.  Adjust the model architecture: Try adding or removing layers, adjusting the number of neurons, or changing the activation functions. For example, you could try using ReLU activation functions in the convolutional layers instead of linear, or reducing the number of neurons in the dense layers to prevent overfitting.
2.  Increase the amount of data: If you have access to more data, you can try increasing the size of your training dataset, which can help improve the model's performance. Alternatively, you could try data augmentation techniques such as rotating, flipping, or shifting the images to generate more training examples.
3.  Tune hyperparameters: Experiment with different hyperparameters such as learning rate, batch size, and regularization rate to see if you can find a combination that works better for your dataset. You can use techniques such as grid search or random search to automate this process.
4.  Use pre-trained models: You can try using pre-trained models such as VGG, ResNet, or Inception to leverage the knowledge gained from training on large datasets like ImageNet. By fine-tuning the last few layers of the pre-trained model, you can adapt it to your specific task and achieve better performance.
5.  Analyze the errors: Analyze the errors made by the model to identify common patterns or types of images that are difficult to classify. This can help guide you towards adjustments to the model architecture or dataset that may improve performance on these difficult cases.

It's important to note that improving the performance of a machine learning model can be an iterative process, and you may need to try multiple strategies before achieving the desired results.

## Hyperparameter Optimisation

In [ ]:
def build_model(hp):
    dense_layer_count = hp.Int('dense_layer_count', min_value=1, max_value=5, step=1)
    dense_layer_neurons = hp.Int('dense_layer_neurons', min_value=32, max_value=128, step=32)
    dense_layer_activation = hp.Choice('dense_layer_activation', values=['linear', 'sigmoid', 'relu'])
    conv_layer_count = hp.Int('conv_layer_count', min_value=1, max_value=5, step=1)
    conv_layer_activation = hp.Choice('conv_layer_activation', values=['linear', 'sigmoid', 'relu'])
    conv_kernel_split = hp.Choice('conv_kernel_split', values=[True, False])
    conv_kernel_size = hp.Int('conv_kernel_size', min_value=3, max_value=7, step=2)
    conv_kernel_stride = hp.Choice('conv_stride', values=[1, 2, 3, 4, 5])
    conv_kernel_count = hp.Int('conv_kernel_count', min_value=16, max_value=64, step=16)
    #learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    optimizer = hp.Choice('optimizer', values=['Adam', 'Adamax', 'Nadam'])
    conv_dropout = hp.Choice('conv_dropout', values=[True, False])
    dense_dropout = hp.Choice('dense_dropout', values=[True, False])

    #for epochs in range(1, 7):
        #print(f'+++++ EPOCHS: {epochs} +++++')
    #Define model structure
    model = tf.keras.models.Sequential()

    stride = (conv_kernel_stride, conv_kernel_stride)
    
    #model.add(data_augmentation_new)
    if conv_kernel_split:
        model.add(Conv2D(conv_kernel_count, kernel_size=(conv_kernel_size, 1), activation=conv_layer_activation, input_shape=(IMG_SIZE,IMG_SIZE,1), padding='same', strides=stride))
        model.add(Conv2D(conv_kernel_count, (1, conv_kernel_size), activation=conv_layer_activation, padding='same', strides=stride))
    else:
        model.add(Conv2D(conv_kernel_count, kernel_size=(conv_kernel_size, conv_kernel_size), activation=conv_layer_activation, input_shape=(IMG_SIZE,IMG_SIZE,1), padding='same', strides=stride))
    
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    
    for i in range(0, conv_layer_count - 1):
        if conv_kernel_split:
            model.add(Conv2D(conv_kernel_count, kernel_size=(conv_kernel_size, 1), activation=conv_layer_activation, padding='same', strides=stride))
            model.add(Conv2D(conv_kernel_count, (1, conv_kernel_size), activation=conv_layer_activation, padding='same', strides=stride))
        else:
            model.add(Conv2D(conv_kernel_count, kernel_size=(conv_kernel_size, conv_kernel_size), activation=conv_layer_activation, padding='same', strides=stride))
        model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    
    if conv_dropout:
        model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    # trial and error: linear performs much better than ReLU and sigmoid
    for i in range(dense_layer_count):
        if i == dense_layer_count - 1 and dense_layer_count > 1:
            if dense_dropout:
                model.add(Dropout(0.2))
        model.add(Dense(dense_layer_neurons, activation=dense_layer_activation))

    # classifier
    model.add(Dense(2, activation='softmax'))
    
    #optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
    #optimizer = keras.optimizers.Adamax(learning_rate = 0.002, beta_1 = 0.9, beta_2 = 0.999)

    model.compile(loss=tf.keras.metrics.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

    return model

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001) # patience 1, min_lr 0.00005
stop_early = EarlyStopping(monitor='val_loss', patience=1)

tuner = BayesianOptimization(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=30
)

history = tuner.search(train_X, train_label, batch_size=128, epochs=15, validation_data=(valid_X, valid_label), callbacks=[reduce_lr, stop_early])